In [1]:
using Serialization, Distributions, AffineInvariantMCMC, StableRNGs 
include("ode_problem.jl")
include("target_probability.jl")
include("define_likelihood_data.jl");

### Name Run

run: 210  
test data: pSHC  
augmented: yes  
AF3 confidence weight: iptm   
prior offset: +/- 2   
binding prior offset: 0  

In [2]:
run = "210"

"210"

Define Prior Distributions

In [3]:
#generate bounds for prior
bound_offset = [2,2]
binding_bound_offset = [0,0]
prior_bounds = return_prior_bounds_empirical(bound_offset, binding_bound_offset) #for target probability
prior_distributions = [Uniform(prior_bounds[i]...) for i in 1:length(prior_bounds)];

Define Test/Train Data

In [4]:
#first input we need are data points and each point's associated standard deviation to evaluate the likelihood
data_points = deserialize("outputs/000_training_data_w_pshc_test.dict")["response"]
std_dev = deserialize("outputs/000_training_data_w_pshc_test.dict")["average_error"]

#load different EGF ligand dosages needed to simulate all training data
ligand_dosages = return_ligand_dose_order_for_likelihood_w_pshc_test()

#provide function for indexing likelihood data - that is, non test data
training_data_indices = likelihood_w_pshc_test;

Define ODEProblems for different intial conditions

In [5]:
#next input we need is an ode problem
odeproblems = []
for egf in ligand_dosages
    odesys, u0, tspan, p = return_ode_problem_default_inputs(egf)
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p) #note p will be overwritten in first run of mcmc algorithm
    push!(odeproblems, prob)
end

Define ODE Solver Inputs

In [6]:
#and ode solver inputs, which are consistent for all simulations
odesolver_inputs = return_ode_problem_solver_default_inputs();

Define Type of ODESolution

In [7]:
#get type of ODEProblem solution
pred = DifferentialEquations.solve(odeproblems[1], odesolver_inputs["solver"], abstol=odesolver_inputs["abstol"], reltol=odesolver_inputs["reltol"], saveat=odesolver_inputs["saveat"])
ode_sol_type = typeof(pred);

Define Augmented Likelihood Term

In [8]:
#also need hyperparameters for augmenting likelihood
augmented_likelihood_parameters = deserialize("outputs/000_augmentation_parameters_predicted_kd_iptm.dict");

Define Target Distribution for Sampling

In [10]:
target_probability = parameters -> logprob_augmented(parameters, prior_distributions, data_points, std_dev, training_data_indices, ligand_dosages, odeproblems, odesolver_inputs, ode_sol_type, augmented_likelihood_parameters)

#51 (generic function with 1 method)

Affine Invariant Sampler Hyperparameters

In [11]:
#affine invariant sampler hyperparameters
numdims = 50
numwalkers = 1000
thinning = 1
numsamples_perwalker = 1000
burnin = 1000
n_subchains = 70
a_burnin = 4
a = 2
rng = StableRNG(28475);

Run Affine Invariant Sampler from 0

In [12]:
x0 = Matrix{Float64}(undef,numdims,numwalkers)
for i in 1:numwalkers
    x0[:,i] = [rand(rng, prior_distributions[j]) for j in 1:numdims]
end

chain0, llhoodvals0 = AffineInvariantMCMC.sample(target_probability, numwalkers, x0, burnin, thinning, a_burnin, rng=rng)
serialize("outputs/$(run)_posterior_samples_subchain_0.jls", chain0)
serialize("outputs/$(run)_posterior_samples_likelihood_subchain_0.jls", llhoodvals0)

#release from RAM
chain0 = nothing
llhoodvals0 = nothing

for i in 1:n_subchains
    chain = deserialize("outputs/$(run)_posterior_samples_subchain_$(i-1).jls")
    chain, llhoodvals = AffineInvariantMCMC.sample(target_probability, numwalkers, chain[:, :, end], numsamples_perwalker, thinning, a, rng=rng)
    serialize("outputs/$(run)_posterior_samples_subchain_$(i).jls", chain)
    serialize("outputs/$(run)_posterior_samples_likelihood_subchain_$(i).jls", llhoodvals)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:13:05
Progress: 100%|█████████████████████████████████████████| Time: 0:15:13
Progress: 100%|█████████████████████████████████████████| Time: 0:15:02
Progress: 100%|█████████████████████████████████████████| Time: 0:15:03
Progress: 100%|█████████████████████████████████████████| Time: 0:15:09
Progress: 100%|█████████████████████████████████████████| Time: 0:14:12
Progress: 100%|█████████████████████████████████████████| Time: 0:14:13
Progress: 100%|█████████████████████████████████████████| Time: 0:14:14
Progress: 100%|█████████████████████████████████████████| Time: 0:14:14
Progress: 100%|█████████████████████████████████████████| Time: 0:14:19
Progress: 100%|█████████████████████████████████████████| Time: 0:14:20
Progress: 100%|█████████████████████████████████████████| Time: 0:14:21
Progress: 100%|█████████████████████████████████████████| Time: 0:14:26
Progress: 100%|█████████████████████████████████████████| Time: 

Run Affine Invariate Sampler from N Subchain

In [11]:
#start = 51
#stop = 70
#for i in start:stop
    #chain = deserialize("outputs/$(run)_posterior_samples_subchain_$(i-1).jls")
    #chain, llhoodvals = AffineInvariantMCMC.sample(target_probability, numwalkers, chain[:, :, end], numsamples_perwalker, thinning, a, rng=rng)
    #serialize("outputs/$(run)_posterior_samples_subchain_$(i).jls", chain)
    #serialize("outputs/$(run)_posterior_samples_likelihood_subchain_$(i).jls", llhoodvals)
#end

Progress:   4%|█▌                                       |  ETA: 0:35:06┌ Warning: At t=0.7235420902632596, dt was forced below floating point epsilon 1.1102230246251565e-16, and step error estimate = 1.3213845949770795. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/Zrnkf/src/integrator_interface.jl:623
┌ Warning: At t=2.2749291839570533, dt was forced below floating point epsilon 4.440892098500626e-16, and step error estimate = 1.3213845951947794. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/Zrnkf/src/integrator_interface.jl:623
┌ Warning: At t=7.723957208811436, dt was forced below floating point epsilon 8.881784197001252e-16, and step error estimate = 1.3